In [1]:
from jupytertracerviz import init_multigpus_repl, multigpus

init_multigpus_repl()

/root/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/root/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install

In [2]:
%%multigpus

import math
from torch import nn
import torch.nn.functional as F
import torch.nn.init as init
from torch.utils.data import Dataset, DataLoader
from torch import distributed as dist
from torch.distributed.tensor import distribute_tensor
from torch.distributed.device_mesh import init_device_mesh
from torch.utils.data.distributed import DistributedSampler
from torch.distributed.fsdp import fully_shard, MixedPrecisionPolicy, CPUOffloadPolicy
from functools import partial
from torch.distributed.algorithms._checkpoint.checkpoint_wrapper import (
    CheckpointImpl,
    apply_activation_checkpointing,
    checkpoint_wrapper,
)
from transformers import (
    set_seed,
    get_linear_schedule_with_warmup,
    AutoConfig,
    AutoTokenizer,
    Glm4MoeForCausalLM,
)
from transformers.models.glm4_moe.modeling_glm4_moe import (
    Glm4MoeMLP,
    Glm4MoeDecoderLayer,
    Glm4MoeTopkRouter,
)
from transformers.models.glm4_moe import modeling_glm4_moe
from liger_kernel.transformers import LigerFusedLinearCrossEntropyLoss
from streaming import LocalDataset
from streaming.base.format.mds.encodings import Encoding, _encodings
import numpy as np

class Model(Glm4MoeForCausalLM):
    def __init__(self, config):
        super().__init__(config)
        self.loss = LigerFusedLinearCrossEntropyLoss(reduction="sum")
        
    def forward(
        self, 
        input_ids, 
        attention_mask=None, 
        position_ids=None, 
        labels=None, 
        num_items_in_batch=None, 
        logits_to_keep=0,
        output_router_logits=None,
        **kwargs,
    ):
        super_out = self.model.forward(
            input_ids = input_ids,
            position_ids = position_ids, 
            attention_mask = attention_mask, 
            output_router_logits=output_router_logits,
            **kwargs,
        )
        if labels is not None:
            embeddings = super_out.last_hidden_state
            slice_indices = slice(-logits_to_keep, None) if isinstance(logits_to_keep, int) else logits_to_keep
            embeddings = embeddings[:, slice_indices, :]
            embeddings = embeddings[:,:-1].reshape(-1, embeddings.shape[-1])
            labels = labels[..., 1:].contiguous()
            labels = labels.reshape(-1)
            
            loss = self.loss(self.lm_head.weight, embeddings, labels)
            print(self.lm_head.weight.shape, embeddings.shape, labels.shape, loss)
            num_items_in_batch = num_items_in_batch.to(loss.device)

            loss = loss / num_items_in_batch
            return {'loss': loss}
        return super_out

class Glm4MoeMoEExpertParallel(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.config = config
        self.device_mesh = self.config.device_mesh
        self.world_size = self.device_mesh.size()
        self.rank = self.device_mesh.get_local_rank()
        
        self.total_experts = config.n_routed_experts
        self.experts_per_rank = self.total_experts // self.world_size
        assert self.total_experts % self.world_size == 0, \
            f"n_routed_experts ({self.total_experts}) must be divisible by world_size ({self.world_size})"
        
        self.local_expert_start = self.rank * self.experts_per_rank
        self.local_expert_end = (self.rank + 1) * self.experts_per_rank
        self._is_sharded = False

        self.experts = nn.ModuleList(
            [
                Glm4MoeMLP(config, intermediate_size=config.moe_intermediate_size)
                for _ in range(config.n_routed_experts)
            ]
        )
        self.gate = Glm4MoeTopkRouter(config)
        self.shared_experts = Glm4MoeMLP(
            config=config, intermediate_size=config.moe_intermediate_size * config.n_shared_experts
        )

    def shard_experts(self):
        if self._is_sharded:
            return

        local_experts = nn.ModuleList([
            self.experts[i] for i in range(self.local_expert_start, self.local_expert_end)
        ])
        del self.experts
        torch.cuda.empty_cache()
        self.experts = local_experts
        self._is_sharded = True

    def moe(self, hidden_states: torch.Tensor, topk_indices: torch.Tensor, topk_weights: torch.Tensor):
        if self._is_sharded:
            return self._moe_sharded(hidden_states, topk_indices, topk_weights)
        else:
            return self._moe_local(hidden_states, topk_indices, topk_weights)
    
    def _moe_local(self, hidden_states: torch.Tensor, topk_indices: torch.Tensor, topk_weights: torch.Tensor):
        hidden_dim = hidden_states.shape[-1]
        final_hidden_states = torch.zeros_like(hidden_states, dtype=hidden_states.dtype)
        
        expert_mask = torch.nn.functional.one_hot(topk_indices, num_classes=len(self.experts)).permute(2, 1, 0)
        expert_hit = torch.greater(expert_mask.sum(dim=(-1, -2)), 0).nonzero().squeeze(-1)
        
        for expert_idx in expert_hit:
            expert_layer = self.experts[expert_idx]
            idx, top_x = torch.where(expert_mask[expert_idx])
    
            current_state = hidden_states[top_x]
            current_hidden_states = expert_layer(current_state) * topk_weights[top_x, idx, None]
            final_hidden_states.index_add_(0, top_x, current_hidden_states.to(hidden_states.dtype))
        
        return final_hidden_states.type(hidden_states.dtype)
    
    def _moe_sharded(self, hidden_states: torch.Tensor, topk_indices: torch.Tensor, topk_weights: torch.Tensor):
        final_hidden_states = torch.zeros_like(hidden_states, dtype=hidden_states.dtype)
        
        expert_mask = torch.nn.functional.one_hot(topk_indices, num_classes=self.total_experts).permute(2, 1, 0)
        
        for local_expert_idx in range(self.experts_per_rank):
            global_expert_idx = self.local_expert_start + local_expert_idx
            expert_layer = self.experts[local_expert_idx]
            
            idx, top_x = torch.where(expert_mask[global_expert_idx])
            
            if top_x.numel() > 0:
                current_state = hidden_states[top_x]
                current_hidden_states = expert_layer(current_state) * topk_weights[top_x, idx, None]
                final_hidden_states.index_add_(0, top_x, current_hidden_states.to(hidden_states.dtype))
            else:
                dummy = expert_layer(hidden_states[:1]) * 0
                final_hidden_states = final_hidden_states + dummy.sum() * 0
                
        torch.distributed.all_reduce(final_hidden_states, group=self.device_mesh.get_group())
        return final_hidden_states.type(hidden_states.dtype)

    def forward(self, hidden_states):
        residuals = hidden_states
        orig_shape = hidden_states.shape
        topk_indices, topk_weights = self.gate(hidden_states)
        hidden_states = hidden_states.view(-1, hidden_states.shape[-1])
        hidden_states = self.moe(hidden_states, topk_indices, topk_weights).view(*orig_shape)
        hidden_states = hidden_states + self.shared_experts(residuals)
        return hidden_states

modeling_glm4_moe.Glm4MoeMoE = Glm4MoeMoEExpertParallel

def check_fn(module):
    return isinstance(module, (Glm4MoeDecoderLayer, Glm4MoeMLP, Glm4MoeMoEExpertParallel))

non_reentrant_wrapper = partial(
    checkpoint_wrapper,
    checkpoint_impl=CheckpointImpl.NO_REENTRANT,
)

class UInt32(Encoding):
    def encode(self, obj) -> bytes:
        return obj.tobytes()

    def decode(self, data: bytes):
        return np.frombuffer(data, np.uint32)

_encodings['uint32'] = UInt32

class Dataset(Dataset):
    def __init__(self, folder):
        self.dataset = LocalDataset(local=folder)
        self.sequence_length = 16384
    
    def __getitem__(self, idx):
        data = self.dataset[idx]
        data.pop('audio', None)
        data.pop('text', None)
        data.pop('token_type_ids', None)

        for k in data.keys():
            data[k] = data[k].astype(np.int64)

        data['labels'] = data['input_ids'].copy()
        return data
    
    def __len__(self):
        return len(self.dataset)

def collator(batch):
    batch = [b for b in batch if b is not None]
    input_ids = [b['input_ids'] for b in batch]
    position_ids = [b['position_ids'] for b in batch]
    labels = [b['labels'] for b in batch]
    attention_mask = [b['attention_mask'] for b in batch]
    input_ids = np.concatenate(input_ids)
    position_ids = np.concatenate(position_ids)
    labels = np.concatenate(labels)
    query_lens = np.concatenate(attention_mask)
    cumsum = [0] + np.cumsum(query_lens).tolist()
    max_cumsum = int(np.max(cumsum))
    cu_seq_lens_q = torch.tensor(cumsum, dtype=torch.int32)
    cu_seq_lens_k = torch.tensor(cumsum, dtype=torch.int32)
    max_seqlen_q = int(np.max(query_lens))
    return {
        'input_ids': torch.tensor(input_ids)[None],
        'position_ids': torch.tensor(position_ids)[None],
        'labels': torch.tensor(labels)[None],
        'cu_seq_lens_q': cu_seq_lens_q,
        'cu_seq_lens_k': cu_seq_lens_k,
        'max_length_q': max_seqlen_q,
        'max_length_k': max_seqlen_q
    }

In [3]:
%%multigpus

class LinearLoRA(nn.Module):
    def __init__(self, linear: nn.Linear, r=4, alpha=1.0):
        super().__init__()
        self.linear = linear
        self.r = r
        self.alpha = alpha
        self.scaling = alpha / r

        in_features = linear.in_features
        out_features = linear.out_features
        
        device = self.linear.weight.device
        dtype = self.linear.weight.dtype

        self.lora_A = nn.ModuleDict({})
        self.lora_B = nn.ModuleDict({})
        
        self.lora_A['e'] = nn.Linear(
            in_features, r, bias=False, 
            device = device,
            dtype = torch.float32,
        )
        self.lora_B['e'] = nn.Linear(
            r, out_features, bias=False, 
            device = device,
            dtype = torch.float32,
        )

        for param in self.lora_A['e'].parameters():
            param.requires_grad = True
        for param in self.lora_B['e'].parameters():
            param.requires_grad = True

        # https://github.com/huggingface/peft/blob/main/src/peft/tuners/lora/layer.py#L260
        init.kaiming_uniform_(self.lora_A['e'].weight, a=math.sqrt(5))
        init.zeros_(self.lora_B['e'].weight)

    def forward(self, x):
        out = self.linear(x)
        lora_update = self.lora_B['e'](self.lora_A['e'](x.to(self.lora_A['e'].weight.dtype))) * self.scaling
        out = out + lora_update.to(x.dtype)
        return out

In [4]:
%%multigpus

import os

rank = int(os.environ["LOCAL_RANK"])
world_size = int(os.environ['WORLD_SIZE'])
device_type = torch.accelerator.current_accelerator()
device = torch.device(f"{device_type}:{rank}")
device_mesh = init_device_mesh(device_type.type, (world_size,), mesh_dim_names=("dp",))
tp_mesh = device_mesh["dp"]
dp_mesh = device_mesh["dp"]
dp_rank = dp_mesh.get_local_rank()
dp_world_size = dp_mesh.size()

model_name = "ramdisk/GLM-4.5-Air"
config = AutoConfig.from_pretrained(model_name)
config.device_mesh = tp_mesh

model = Model.from_pretrained(
    model_name, 
    attn_implementation="kernels-community/vllm-flash-attn3",
    torch_dtype=torch.bfloat16,
    config=config,
)

In [5]:
%%multigpus

for module in model.modules():
    if isinstance(module, Glm4MoeMoEExpertParallel):
        module.shard_experts()

In [6]:
%%multigpus

from tqdm import tqdm

for name, param in model.named_parameters():
    param.requires_grad = False

selected = [
    "q_proj", 
    "k_proj", 
    "v_proj", 
    "o_proj",
    "gate_proj",
    "up_proj",
    "down_proj"
]

rank_lora = 256
alpha_lora = 512
top_k = model.config.num_experts_per_tok
r = rank_lora // top_k
alpha = alpha_lora // top_k

for name, module in tqdm(model.named_modules()):
    for child_name, child in module.named_children():
        if len(child_name) and any([a in child_name for a in selected]) and isinstance(child, nn.Linear):
            if 'mlp.experts' in name:
                selected_rank = r
                selected_alpha = alpha
            else:
                selected_rank = rank_lora
                selected_alpha = alpha_lora

            lora = LinearLoRA(child, r=selected_rank, alpha=selected_alpha)
            setattr(module, child_name, lora)

In [7]:
%%multigpus

ignored_params = set()
for module in model.modules():
    if isinstance(module, Glm4MoeMoEExpertParallel):
        for param in module.experts.parameters():
            ignored_params.add(param)

In [8]:
%%multigpus

lora_mp_policy = MixedPrecisionPolicy(
    param_dtype=torch.float32,
    reduce_dtype=torch.float32,
)

for name, module in model.named_modules():
    for child_name, child in module.named_children():
        if '.lora' in name and isinstance(child, nn.Linear):
            fully_shard(child, mp_policy=lora_mp_policy)

In [9]:
%%multigpus

fsdp_kwargs = {}
fsdp_kwargs["mp_policy"] = MixedPrecisionPolicy(
    param_dtype=torch.bfloat16,
    reduce_dtype=torch.float32,
)
fsdp_kwargs["mesh"] = dp_mesh
fsdp_kwargs["ignored_params"] = ignored_params

for module in model.modules():
    if isinstance(module, Glm4MoeDecoderLayer):
        fully_shard(module, **fsdp_kwargs)

fully_shard(model, **fsdp_kwargs)

In [10]:
%%multigpus

for module in model.modules():
    if isinstance(module, Glm4MoeMoEExpertParallel):
        module.experts = module.experts.to(device)

In [11]:
%%multigpus

apply_activation_checkpointing(
    model,
    checkpoint_wrapper_fn=non_reentrant_wrapper,
    check_fn=check_fn,
)

In [12]:
# %%multigpus

# dataset = Dataset('multipacking-glm')

# b = [dataset[0], dataset[1]]
# b = collator(b)
# with torch.no_grad():
#     for k in b.keys():
#         if isinstance(b[k], torch.Tensor):
#             b[k] = b[k].to(model.device, non_blocking=True)

#     valid_tokens = (b['labels'] != -100).sum().item() * dp_world_size
#     b['num_items_in_batch'] = torch.tensor(valid_tokens)
#     out = model(**b, use_cache=False)

# out

In [13]:
%%multigpus

optim = torch.optim.AdamW(model.parameters(), lr=1e-4, fused=True, weight_decay=0.01)

In [14]:
%%multigpus

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [15]:
%%multigpus

dataset = Dataset('multipacking-glm')
sampler = DistributedSampler(
    dataset,
    num_replicas=dp_world_size,
    rank=dp_rank,
    shuffle=True,
)
train_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=2,
    num_workers=5,
    prefetch_factor=5,
    pin_memory=True,
    collate_fn=collator,
)
iter_train_loader = iter(train_loader)

In [16]:
# %%multigpus

# strings = [
#     "hello what is your name",
#     "testing testing 123",
#     'Perdana Menteri, Datuk Seri Anwar Ibrahim menjelaskan ketidakhadirannya ke sidang Dewan Rakyat pagi tadi kerana menghadiri sesi taklimat khas Majlis Raja-Raja Melayu di Istana Negara yang membincangkan pelbagai isu semasa dan urusan rasmi negara.'
# ]
# mapping = {}
# for s in strings:
#     messages = [{"role": "user", "content": s}]
#     inputs = tokenizer.apply_chat_template(
#         messages,
#         tokenize=True,
#         add_generation_prompt=True,
#         return_dict=True,
#         return_tensors="pt",
#     )
#     inputs.pop('token_type_ids')
#     with torch.no_grad():
#         inputs = inputs.to(device)
#         o = model(**inputs, use_cache=False)
#         l = model.lm_head(o.last_hidden_state)
#         mapping[s] = l.cpu()

In [17]:
# %%multigpus

# here I already verified
# mapping_logits = torch.load('moe-logits.pkl')
# for k, v in mapping_logits.items():
#     print(k, (v - mapping[k]).abs().max())

In [18]:
%%multigpus

@torch.no_grad()
def clip_grad_norm_(parameters, max_norm, norm_type=2.0):
    if isinstance(parameters, torch.Tensor):
        parameters = [parameters]
    else:
        parameters = list(parameters)
    
    grads = [p.grad for p in parameters if p.grad is not None]
    
    norms = []
    for grad in grads:
        if hasattr(grad, 'full_tensor'):  # DTensor
            grad_full = grad.full_tensor()
        else:
            grad_full = grad
        
        if norm_type == float('inf'):
            norms.append(grad_full.abs().max())
        else:
            norms.append(grad_full.norm(norm_type))
    
    if len(norms) == 0:
        return torch.tensor(0.0)
    
    total_norm = torch.stack(norms).norm(norm_type)
    
    clip_coef = max_norm / (total_norm + 1e-6)
    clip_coef_clamped = torch.clamp(clip_coef, max=1.0)
    
    for grad in grads:
        if hasattr(grad, 'full_tensor'):  # DTensor
            grad.mul_(clip_coef_clamped)
        else:
            grad.mul_(clip_coef_clamped)
    
    return total_norm
    
for i in range(5):
    total_tokens = 0
    b = next(iter_train_loader)
    for k in b.keys():
        if isinstance(b[k], torch.Tensor):
            b[k] = b[k].to(device, non_blocking=True)

    valid_tokens = (b['labels'] != -100).sum().item()
    total_tokens += valid_tokens
    token_tensor = torch.tensor([total_tokens], dtype=torch.long, device=device)
    dp_group = dp_mesh.get_group()
    dist.all_reduce(token_tensor, op=dist.ReduceOp.SUM, group=dp_group)
    global_total_tokens = token_tensor.item()
    b['num_items_in_batch'] = torch.tensor(global_total_tokens)
    out = model(**b, use_cache=False)
    loss = out["loss"] * dp_world_size
    print('done forward')
    loss.backward()
    grad_norm = clip_grad_norm_(model.parameters(), 1.0)
    optim.step()
    optim.zero_grad()

    print(i, loss, out["loss"])

[GPU 5] torch.Size([151552, 4096]) torch.Size([32697, 4096]) torch.Size([32697]) tensor(136561.2344, device='cuda:5',
       grad_fn=<LigerFusedLinearCrossEntropyFunctionBackward>)
[GPU 5] done forward
[GPU 1] torch.Size([151552, 4096]) torch.Size([32697, 4096]) torch.Size([32697]) tensor(136561.2344, device='cuda:1',
       grad_fn=<LigerFusedLinearCrossEntropyFunctionBackward>)
[GPU 1] done forward
[GPU 0] torch.Size([151552, 4096]) torch.Size([32697, 4096]) torch.Size([32697]) tensor(136561.2344, device='cuda:0',
       grad_fn=<LigerFusedLinearCrossEntropyFunctionBackward>)
[GPU 0] done forward
[GPU 6] torch.Size([151552, 4096]) torch.Size([32697, 4096]) torch.Size([32697]) tensor(136561.2344, device='cuda:6',
       grad_fn=<LigerFusedLinearCrossEntropyFunctionBackward>)
[GPU 7] torch.Size([151552, 4096]) torch.Size([32697, 4096]) torch.Size([32697]) tensor(136561.2344, device='cuda:7',
       grad_fn=<LigerFusedLinearCrossEntropyFunctionBackward>)
[GPU 6] done forward
[GPU 7] don